# Web Scraping de la pagina Agencia Meteorologica de Japon

In [117]:
#Importamos todas las librerias necesarias para realizar el Web Scrapping

import collections
collections.Callable = collections.abc.Callable
from bs4 import BeautifulSoup as soup
import selenium
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import requests
import time
import schedule
import ydata_profiling 
from ydata_profiling import ProfileReport


Con la libreria Selenium, abriremos un navegador Chrome, en la pagina de los anuncios de Yapo

In [118]:
browser = webdriver.Chrome() #Cargamos el driver a una variable
myUrl = "https://www.data.jma.go.jp/multi/quake/?lang=es" #Url de la pagina donde vamos a ingresar
browser.get(myUrl) #Abrimos Chrome
time.sleep(5)
pageSoup = soup(browser.page_source, "html.parser") #Convertimos nuestro archivo a HTML.
pages = pageSoup.find("table", class_="quakeindex_table") #Encontramos el tag y la clase que nos interesa obtener.
links = pages.find_all("a") #Encontramos el tag.

In [ ]:
links #Revisamos que realizo la extraccion de los URL

Extraemos los links actualizados y separados

In [120]:
lista_links = [] #Lista vacia para almacenar los links
for indice, elemento in enumerate(links): #iteramos en los links
    if "quake" in str(links[indice]): 
        inicio = str(links[indice]).index("quake") #Extraemos el inicio del link
        fin = str(links[indice]).index(">")-1 #extraemos el final del link
        link = str(links[indice])[inicio:fin] #Obtenemos los links
        link = str(link).replace("amp;", "") #Reemplazamos para obtener el link limpio
        lista_links.append(link) #Agregamos a la lista vacia

In [ ]:
lista_links #Revisamos que realizo la adiccion de los links

Extraemos la informacion necesaria por cada link y reemplazamos la informacion necesaria para que quede limpio e ingestarlo en nuestro Dataframe

In [ ]:
#Creamos la tabla donde ira la informacion
df = pd.DataFrame(0, index=range(len(lista_links)),columns=["Hora y dia del Sismo", "Latitud", "Longitud", "Magnitud", "Profundidad del Hipocentro", "Lugar del Epicentro"])

seguir = 0
comodin = 0
while seguir != len(lista_links):
    lista_links = [] #Lista vacia para almacenar los links
    for indice, elemento in enumerate(links): #iteramos en los links
        if "quake" in str(links[indice]): 
            inicio = str(links[indice]).index("quake") #Extraemos el inicio del link
            fin = str(links[indice]).index(">")-1 #extraemos el final del link
            link = str(links[indice])[inicio:fin] #Obtenemos los links
            link = str(link).replace("amp;", "") #Reemplazamos para obtener el link limpio
            lista_links.append(link) #Agregamos a la lista vacia
    seguir += 1
  
    while comodin != len(lista_links):
        datos = [] #Lista vacia para almacenar los datos de cada fecha
        for indice in range(len(lista_links)): #iteramos sobre la lista de links
            url_unico = "https://www.data.jma.go.jp/multi/quake/"+lista_links[indice] #concatenamos la informacionn de cada elemento con el url.
            browser_2 = webdriver.Chrome() #Cargamos el driver a una variable
            myUrl = url_unico #Url de Yapo para venta de celulares en la region metropolitana
            browser_2.get(myUrl) #Abrimos Chrome
            time.sleep(7) 
            pageSoup = soup(browser_2.page_source, "html.parser") #Convertimos nuestro archivo a HTML.
            pages = pageSoup.find("table", class_="quakeindex_table") #Encontramos el tag y la clase que nos interesa obtener.
            tr = pages.find_all("td")  #Encontramos el tag.
            datos.append(tr) #Agregamos a la lista vacia
        
            for indice, elemento in enumerate(datos): #iteramos para reemplazar las tags que trae del web scraping.
                if len(datos[indice]) == 6:
                    df["Hora y dia del Sismo"][indice] = str(datos[indice][0]).replace("</td>", "").replace("<td>", "")
                    df["Latitud"][indice] = str(datos[indice][1]).replace("</td>", "").replace("<td>", "")
                    df["Longitud"][indice] = str(datos[indice][2]).replace("</td>", "").replace("<td>", "")
                    df["Magnitud"][indice] = str(datos[indice][3]).replace("</td>", "").replace("<td>", "")
                    df["Profundidad del Hipocentro"][indice] = str(datos[indice][4]).replace("</td>", "").replace("<td>", "")
                    df["Lugar del Epicentro"][indice] = str(datos[indice][5]).replace("</td>", "").replace("<td>", "")
                else:
                    print(indice, elemento)
        comodin+=1


In [124]:
df#Verificamos que se haya scrapeado la info.

,Hora y dia del Sismo,Latitud,Longitud,Magnitud,Profundidad del Hipocentro,Lugar del Epicentro
0,2023/06/30 00:48,36.5N,140.7E,4.0,50km,Alta mar de Prefectura de Ibaraki
1,2023/06/29 22:20,24.8N,124.2E,3.8,80km,Mares cercanos de Isla Ishigaki
2,2023/06/29 20:01,28.4N,129.5E,3.1,30km,Mares cercanos de Islas de Amamioshima
3,2023/06/29 02:27,35.0N,135.6E,2.2,Poco profundo,Sur de Prefectura de Kioto
4,2023/06/28 17:22,44.9N,142.1E,3.0,10km,Norte de región Soya
...,...,...,...,...,...,...
163,2023/05/31 15:20,33.8N,135.0E,3.1,10km,Canal de Kii
164,2023/05/31 10:04,38.9N,142.0E,3.7,50km,Alta mar de Prefectura de Miyagi
165,2023/05/31 07:49,33.4N,139.3E,4.3,10km,Mares cercanos de Isla de Hachijyo
166,2023/05/31 04:06,36.0N,137.4E,2.8,10km,"Región Hida, Prefectura de Gifu"
